In [4]:
import pathlib
import random
from typing import List
import numpy as np
data_label = list() 

'''turn string into 0 and 1
'''
def demap(_list: List):
    final = list()
    final.append(float(_list[0]))
    mid_term = [0.01 if _l=='None' else 0.99 for _l in _list[1:]]
    final.extend(mid_term)
    return final

def reform_data(inputlist: List):
    _l = int(len(inputlist) / 10)
    len_input = _l * 10
    return inputlist[:len_input]

'''split dataset into samples for model training
'''

def data_split(inputlist: List):
    len_samples = int (len(inputlist) / 10)
    final_samplelist = list()

    for i in range(len_samples):
        sample = inputlist[10*i:10*(i+1)]
        final_samplelist.append(sample)

    return final_samplelist

def mix_datasets(
    input_data: List,
    labels: List
):
    final_output = list()
    '''change the datalist into list of dict, then shuffle
    '''

    for _data, _label in zip(input_data, labels):
        data = {
            tuple(_data): _label
        }
        final_output.append(data)

    return final_output

def test_data_gen(
    _l: List,
    percentage: List
):
    return _l[int(len(_l)*percentage[0]): int(len(_l)*percentage[1])]

'''label of all data, whole sample data
'''

labels = list() 
final_samples = list()

sample_list = list()

test_list = list()
test_labels = list()


for a in range (1,100):
    with open (f'../datafile/datafile{a}.txt') as _f:
        data = _f.read().split('\n')
    data_label.append(data[0])
    data = demap(data[:-2])

    real_data = data[1:]
    len_real_data = len(real_data)

    # print (f'''type of len: {
    #     type(len_real_data)}\ntype of data[0]: {type(data[0])}
    # ''')
    failure_node = int(len_real_data * data[0])
    

    normal_data = real_data[:failure_node]
    fading_data = real_data[failure_node:]

    test_list.extend(data_split(reform_data(normal_data)))
    test_labels.extend([0] * len(reform_data(normal_data)))

    _fail = reform_data(
            test_data_gen(
                fading_data,
                [0, 0.1]
            )
        )
    
    test_list.extend(
        data_split(
            _fail
        )   
    )

    test_labels.extend(
        [1] * len(
            reform_data(
                _fail
            )
        )    
    )


    # print (fading_data)
    normal_data = normal_data[:len(fading_data)] # make the dataset balanced
    '''make the data consisted of data having the integer number of hundreds

    Label the datasets and create the whole dataset
    for training and testing
    '''

    normal_data_list = reform_data(normal_data)
    fading_data_list = reform_data(fading_data)
    normal_samplelist = data_split(normal_data_list)
    fading_samplelist = data_split(fading_data_list)


    '''label the data
    '''

    sample_list.extend(normal_samplelist)
    sample_list.extend(fading_samplelist)
        
    labels.extend([0] * len(normal_samplelist))
    labels.extend([1] * len(fading_samplelist))


    


final_list = mix_datasets(sample_list, labels)

#
test_list = mix_datasets(test_list, test_labels)


In [5]:


from sklearn.model_selection import train_test_split

'''demap the dict list
'''


data_sets = list()
train_datasets = final_list[:int(len(final_list) * 0.9)]
random.shuffle(train_datasets)

random.shuffle(test_list)

def fig_extract(
    final_list: List
):
    data_sets = list()
    for _key in final_list:
        data_sets.extend(list(_key.keys()))
    data_sets = np.array(data_sets)

    data_labels = list()

    for _k in final_list:
        data_labels.extend(list(_k.values()))
    
    data_labels = np.array(data_labels)

    return data_sets, data_labels

X_train, y_train = fig_extract(train_datasets)

X_test, y_test =  fig_extract(test_list)




In [7]:
X_test

array([[0.99, 0.99, 0.99, ..., 0.99, 0.99, 0.99],
       [0.99, 0.99, 0.99, ..., 0.99, 0.99, 0.99],
       [0.99, 0.99, 0.99, ..., 0.99, 0.99, 0.99],
       ...,
       [0.99, 0.99, 0.99, ..., 0.99, 0.99, 0.99],
       [0.99, 0.99, 0.99, ..., 0.99, 0.99, 0.99],
       [0.99, 0.99, 0.99, ..., 0.99, 0.99, 0.99]])

In [ ]:


from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM


from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb

max_features = 20000
# 在此数量的单词之后剪切文本（取最常见的 max_features 个单词）
maxlen = 10
batch_size = 16

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(X_train, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 64))
model.add(LSTM(10, dropout=0.5, recurrent_dropout=0.2))
model.add(Dense(1, activation='relu'))

# 尝试使用不同的优化器和优化器配置
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=15,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

array([[0.01, 0.99, 0.99, ..., 0.99, 0.99, 0.99],
       [0.99, 0.99, 0.99, ..., 0.99, 0.99, 0.99],
       [0.99, 0.99, 0.99, ..., 0.99, 0.99, 0.99],
       ...,
       [0.99, 0.99, 0.99, ..., 0.01, 0.99, 0.99],
       [0.99, 0.99, 0.99, ..., 0.99, 0.99, 0.99],
       [0.01, 0.01, 0.01, ..., 0.01, 0.01, 0.01]])